<a href="https://colab.research.google.com/github/Exar2004/FATEC_MINERACAO_DADOS_2SEM2024/blob/main/Murillo%20lazarini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ATIVIDADE P1**

## Análise Exploratória de Dados

Links deste exercício:
- https://www.kaggle.com/datasets/zynicide/wine-reviews
- https://seaborn.pydata.org/
- https://www.wine-searcher.com/critics-17-wine+enthusiast

Fomos contratados por uma grande empresa distribuidora de vinhos. Nosso cliente deseja melhorar as vendas entregando produtos que agradem os diversos perfis de clientes. O foco do nosso projeto aqui é desenvolver uma análise exploratória nos [dados de avaliações de vinhos](https://www.kaggle.com/datasets/zynicide/wine-reviews).

Uma análise exploratória bem elaborada consiste em algumas etapas:

- Compreensão do negócio
- Compreensão dos dados
- Preparar os dados (limpeza e tratamento)
- Análise de dados (extrair informações relevantes e responder perguntas)
- Avaliação dos resultados

Nosso objetivo ao concluir esse projeto é responder as seguintes perguntas:

1. De onde vem o melhor vinho ?
2. O preço do vinho está relacionado com a qualidade ?
3. Certas variedades de uvas são de maior qualidade ?
4. Qual é o melhor vinho em cada faixa de preço ?
5. Que palavras são mais utilizadas para descrever um vinho ?

Vamos começar!

# *Verificando os dados*

Agora que já sabemos que perguntas devemos responder durante a análise, vamos dar uma olhada nos dados. Começaremos com alguns imports

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pit
import seaborn as sns
sns.set()

Utilizaremos a biblioteca Seaborn, uma opção de biblioteca para visualização baseada na matplotlib que fornece uma interface mais simples para plotar os gráficos (escreveremos menos código).


Vamos examinar o dataset:

In [6]:
df = pd.read_csv("/winemag-data-130k-v2.csv")
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


Mostrando somente os nomes das colunas

In [8]:
df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

Vamos elaborar um pequeno sumário para essas colunas e ver o que iremos utilizar em nossa análise:

*   unnamed - parece ser uma coluna de índice que podemos remover
*   country - país onde o vinho foi feito (interessante)
*   designation - vinhedo dentro da adega de onde as uvas que fizeram o vinho foram coletadas.
*   points - é o número de pontos que o vinho obteve em sua avaliação (precisamos entender a escala de pontos)
*   price - preço da garrafa de vinho
*   province - é a província (estado) onde o vinho foi feito
*   region_1 - é a região onde o vinho foi feito
*   region_2 - região mais específica dentro de uma vitícola (podemos remover isso)
*   taster_name - nome do revisor
*   taster_twitter_handle - perfil do Twitter do avaliador
*   title - nome do vinho analisado (o ano de fabricação está no título)
*   variety - tipo de uvas utilizadas na fabricação do vinho
*   winery - adega onde o vinho foi feito.

Agora, uma olhada rápida nos tipos de dados:

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


Ao todo temos 129.971 linhas e 14 colunas no dataset. Existem algumas colunas com dados ausentes e podemos descartar algumas dessas colunas, como por exemplo region_2, taster_name e taster_twitter_handle

Vamos fazer uma pré análise estatística dos valores numéricos de nosso dataframe com o comando describe()

In [10]:
df.describe()

,Unnamed: 0,points,price
count,129971.000000,129971.000000,120975.000000
mean,64985.000000,88.447138,35.363389
std,37519.540256,3.039730,41.022218
min,0.000000,80.000000,4.000000
25%,32492.500000,86.000000,17.000000
50%,64985.000000,88.000000,25.000000
75%,97477.500000,91.000000,42.000000
max,129970.000000,100.000000,3300.000000


O describe() nos mostra uma distribuição de pontos para uma garrafa de vinho, onde o minimo é 80 e o máximo é 100. Também é possível notar que a garrafa de vinho mais barata custa 4 e a mais cara $3.300. Segundo esse site temos a seguinte classificação por pontos:
*   98–100 – Classic
*   94–97 – Superb
*   90–93 – Excellent
*   87–89 – Very good
*   83–86 – Good
*   80–82 – Acceptable


Por meio da função value_counts() podemos contar quantos vinhos foram avaliados por cada país:




In [11]:
df["country"].value_counts()

,count
country,
US,54504
France,22093
Italy,19540
Spain,6645
Portugal,5691
Chile,4472
Argentina,3800
Austria,3345
Australia,2329


Note que a maioria dos vinhos avaliados é dos EUA, França, Itália, Espanha e Portugal.

Também podemos utilizar value_counts() para verificar quais tipos de uva foram mais utilizadas na fabricação dos vinhos avaliados:

In [12]:
df["variety"].value_counts()

,count
variety,
Pinot Noir,13272
Chardonnay,11753
Cabernet Sauvignon,9472
Red Blend,8946
Bordeaux-style Red Blend,6915
...,...
Cabernet Sauvignon-Barbera,1
Sauvignonasse,1
Forcallà,1


Nossa próxima tarefa é limpar os dados, agora que verificamos podemos concluir que os campos: Unamed: 8, designation, region_1, region_2, taster_name e taster_twitter_handle podem ser removidos, pois não fornecem dados úteis para nossa análise.

# Linpando os dados